# load libraries and data

In [1]:
from torch import nn
import torch
from torch import tensor 
from torch.autograd import Variable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from sklearn.metrics import roc_auc_score
"""k fold 대신 stratified k fold"""
from sklearn.model_selection import StratifiedKFold
import torch
import itertools
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm
from pathlib import Path
import os

# psychosocial + rsfMRI + strurctural MRI + diffusion MRI


dataset_name = 'rsf_task3fMRI'
train_out = Path(os.getcwd()+'/Data/sa_ppc_'+dataset_name+'_train.csv')
test_out =Path(os.getcwd()+'/Data/sa_ppc_'+dataset_name+'_test.csv')


train_data= pd.read_csv(train_out)
test_data= pd.read_csv(test_out)


target ='SuicideAttempt'
unused_feat = ['abcd_site','kfold']

# mri feature가 시작하는 column의 index 구하기
# np.where의 결과값이 array에 들어가기 때문에 방금 계산해 넣어놓은 [0]번째 값을 가져온다.
start_psycho_index = np.where(test_data.columns.values == "race.ethnicity_1")[0][0]
start_rsfmri_index = np.where(test_data.columns.values == "rsfmri_var_cort.destrieux_g.and.s.frontomargin.lh")[0][0]

start_sstfmri_index = np.where(test_data.columns.values == "tfmri_sst_all_correct.go.vs.fixation_beta_cort.destrieux_g.and.s.frontomargin.lh")[0][0]
start_Nbackfmri_index = np.where(test_data.columns.values == "tfmri_nback_all_0.back_beta_cort.destrieux_g.and.s.frontomargin.lh")[0][0]
start_midfmri_index = np.where(test_data.columns.values == "tfmri_mid_all_antic.reward.vs.neutral_beta_cort.destrieux_g.and.s.frontomargin.lh")[0][0]


psychosocial = list(test_data.columns[start_psycho_index:start_rsfmri_index])
rsf_mri = list(test_data.columns[start_rsfmri_index:start_sstfmri_index])
sstfmri = list(test_data.columns[start_sstfmri_index:start_Nbackfmri_index])
Nbackfmri = list(test_data.columns[start_Nbackfmri_index:start_midfmri_index])
midfmri = list(test_data.columns[start_midfmri_index:])


Num_FOLDS  = 5
# the number of feature that you want to show 
Num_feat = 20


print(len(train_data), len(train_data.columns))
print(len(psychosocial))
print(len(rsf_mri))
print(len(sstfmri))
print(len(Nbackfmri))
print(len(midfmri))

4614 3979
129
148
1036
1332
1330


In [3]:
target ='SuicideAttempt'


In [ ]:
#train_data['subjectkey']
#test_data['subjectkey']

# Fine object featrues

# For getting feature importance 

In [4]:
def feature(Num_feat, clf_res, test_data_processed, features):
    importance_res = []
    for i in clf_res:
        importance_clf =i.feature_importances_
        importance_res.append(importance_clf)
    
    importance=[importance_res[0][i]/5+importance_res[1][i]/5+ importance_res[2][i]/5+ importance_res[3][i]/5+ importance_res[4][i]/5 for i in range(len(importance_res[1]))]
    
    #print(importance)
    #plt.plot(importance)
    #plt.show()

    #importance_sort = importance.sort(reverse=True)
    
    #labels_importance=importance.argsort()[::-1]

    #feat_name_sort=test_data_processed[features].columns[labels_importance]
    feat_name_sort = test_data_processed[features].columns
    important_features = pd.DataFrame([importance],columns = feat_name_sort, index=['Importance']) 
    important_features =important_features.transpose().sort_values(by=['Importance'], ascending=False)
    important_features = important_features.head(50)
    """
    for i in range (len(test_data_processed[features].columns)):
        feature = pd.DataFrame([[feat_name_sort[i],importance_sort[i]]], columns = ['feature name', 'ratio'])
        important_features=pd.concat([important_features,feature])
    """
    return important_features

# Define function for preprocessing for Cross validation

In [5]:
def preprocessing (train_data, test_data, NUM_FOLDS):
    test_data_processed= test_data.fillna(0).reset_index(drop=True)
    train_data_processed = train_data.fillna(0).reset_index(drop=True)
    
    # 추가? 값 변경?
    test_data_processed["kfold"] = -1
    train_data_processed["kfold"] = -1

    # frac: 전체 row 중 몇 %를 반환할 지 결정 -> frac=1을 설정해서 모든 데이터를 반환
    # random_state: 추후 이것과 동일한 샘플링을 재현하기 위함
    # sample: 데이터에서 임의의 샘플 선정 -> frac=1이면 전체 data의 순서만 임의로 바뀜
    train_data_processed = train_data_processed.sample(frac=1,random_state=2020).reset_index(drop=True)
    
    """stratified로 수정"""
    # kf = KFold(n_splits=NUM_FOLDS) 
    kf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state =0)
    
    # enumerate: 각 split된 data set 순서대로 index를 함께 반환
        # ex. fold 0 일 때, 80%는 trn_, 20%는 val_
    for fold, (trn_, val_) in enumerate(kf.split(X=train_data_processed, y=train_data_processed[target])):
        #print(len(trn_), len(val_)) -> 출력: 4408, 1102
        # 'kfold' 칼럼은 cross validation할 때 fold 순서를 지정해놓은 것. 
        train_data_processed.loc[val_, 'kfold'] = fold
    
    print("done preprocessing")
    return train_data_processed, test_data_processed

# Finding best parameters

In [6]:
# Augmented
import torch
import itertools
from sklearn.metrics import confusion_matrix
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm


def find_bestpar(fold, train_data_processed, test_data_processed, features):
    
    """test data 생성"""
    X_test = test_data_processed[features].values
    Y_test = test_data_processed[target].values
    
    # Store maximum auc
    max_auc= 0
    # Store maximum hypterparameter set
    max_hy = []
    """
    # define hyperparameter space : learning rate, 
    n_ = [4,8,16]                              # 
    lr_ = [2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 1e-4] # learning rate
    w_ = [0.01, 0.001, 0.0001]                 # weight decay
    g_ = [0.95, 0.99, 0.9]                     # scheduler params - gamma
    ss_ = [10, 20, 30]                         # scheduler params - step_size
    
    # Orginal hyperparameter space 
    """
    # define hyperparameter space (quick version)
    n_ = [4,16]
    lr_ = [2e-2,1e-3]
    w_ = [0.01,0.001]
    g_ = [0.95,0.99]
    ss_ = [10,30]
    
    
    
    
    all_ = [n_, lr_, w_, g_, ss_]
    h_space = [s for s in itertools.product(*all_)]
    
    for hy in tqdm(h_space):
        """===================Cross Validation==================="""
        
        """validation & test 결과"""
        valid_res = []
        test_auc_res = []
        test_acc_res = []
     
        for i in range(fold):
            #print("fold ", i)
            # 5개의 fold 사용했으므로 변수 fold 값은 차례대로 0,1,2,3,4 중 하나
            clf = TabNetClassifier(n_a = hy[0],
                                n_d = hy[0],
                                optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                                scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                verbose=0)
            
            df_train = train_data_processed[train_data_processed.kfold != i]  # 5개 중 4개 train에 할당
            df_valid = train_data_processed[train_data_processed.kfold == i]  # 5개 중 1개 validation에 할당
            
            X_train = df_train[features].values
            Y_train = df_train[target].values
            
            X_valid = df_valid[features].values
            Y_valid = df_valid[target].values
            
            


            clf.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                    eval_name=['train', 'valid'], eval_metric=['auc'],
                    max_epochs=200 , patience=20)
       
            preds_acc = clf.predict(X_test)
            preds_prob = clf.predict_proba(X_test)
            test_auc = roc_auc_score(y_score=preds_prob[:,1], y_true=Y_test)
            test_acc = accuracy_score(preds_acc, Y_test)
            
            valid_res.append(clf.best_cost)
            test_auc_res.append(test_auc)
            test_acc_res.append(test_acc)
            print('[%3d/%4d] '%(i+1, fold),'Valid score: %2f'% clf.best_cost, 'Test AUC: %.3f%%'%test_auc, 'Test ACC: %.3f%%'%test_acc)
    
        #print(valid_res)
        #print(test_auc_res)
        #print(test_acc_res)
        """valid와 test의 평균, 표준편차 출력"""
        print("=====parameter별 valid, test score=====")
        print("Validation 평균: %3f"%np.mean(valid_res), "Test AUC 평균: %3f"%np.mean(test_auc_res), "Test ACC 평균: %3f"%np.mean(test_acc_res))

        if np.mean(test_auc_res)>max_auc:
            print("Find new maximum AUC!!")
            max_hy = hy
            max_auc = np.mean(test_auc_res)
    
    return max_hy


# Train with best parameter

In [7]:
def bestpar_tuning(fold, train_data_processed, test_data_processed, max_hy, features):
    hy = max_hy
    print("Max hy:" ,hy)
    X_test = test_data_processed[features].values
    Y_test = test_data_processed[target].values
    """validation & test 결과"""
    valid_res = []
    test_auc_res = []
    test_acc_res = []
    clf_res = []
    preds_prob_res = []
    
    """해당 버전에 추가된 코드"""    
    y_valid_true = []
    y_valid_pred = []
    y_test_pred = []
    
    y_valid_subject = []
    y_test_subject = []
    
    
    for i in range(fold):
        clf = TabNetClassifier(n_a = hy[0],n_d = hy[0],
                           optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                           scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                           scheduler_fn=torch.optim.lr_scheduler.StepLR,
                           verbose=0)
        
        # 5개의 fold 사용했으므로 변수 fold 값은 차례대로 0,1,2,3,4 중 하나
        df_train = train_data_processed[train_data_processed.kfold != i]  # 5개 중 4개 train에 할당
        df_valid = train_data_processed[train_data_processed.kfold == i]  # 5개 중 1개 validation에 할당
            
        X_train = df_train[features].values
        Y_train = df_train[target].values
            
        X_valid = df_valid[features].values
        Y_valid = df_valid[target].values
        
        #print(X_valid)
        # 학습 전 subject key 가져옴
        y_valid_subject.append(df_valid['subjectkey'].values)
        y_test_subject.append(test_data_processed['subjectkey'].values)  
        """해당 버전 추가 코드"""
        y_valid_true.append(Y_valid)        
        
        # 학습
        clf.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                        eval_name=['train', 'valid'], eval_metric=['auc'],
                        max_epochs=200 , patience=20)
        
        # 결과들
        """fold별 validation & test data에 대한 target 예측 배열"""
        """해당 버전 추가 코드"""
        preds_prob_val= clf.predict_proba(X_valid)[:,1]
        y_valid_pred.append(preds_prob_val)
        # y_valid_pred.append(clf.predict(X_valid))
        y_test_pred.append(clf.predict(X_test))


        preds_acc = clf.predict(X_test)
        preds_prob = clf.predict_proba(X_test)
        test_auc = roc_auc_score(y_score=preds_prob[:,1], y_true=Y_test)
        test_acc = accuracy_score(preds_acc, Y_test)

                    
        valid_res.append(clf.best_cost)
        test_auc_res.append(test_auc)
        test_acc_res.append(test_acc)
        print('[%3d/%4d] '%(i+1, fold),'Valid score: %2f'% clf.best_cost, 'Test AUC: %.3f'%test_auc, 'Test ACC: %.3f'%test_acc)
        preds_prob_res.append(preds_prob)
        clf_res.append(clf)
    
    """valid와 test의 평균, 표준편차 출력"""
    print("Validation 평균: %3f"%np.mean(valid_res), "Test AUC 평균: %3f "%np.mean(test_auc_res), "Test ACC 평균: %3f"%np.mean(test_acc_res))
    
    preds_prob=[preds_prob_res[0][i]/5+preds_prob_res[1][i]/5+ preds_prob_res[2][i]/5+ preds_prob_res[3][i]/5+ preds_prob_res[4][i]/5 for i in range(len(preds_prob_res[1]))]
    preds_prob = np.array(preds_prob)
    valid_result = np.mean(valid_res)
    test_auc = np.mean(test_auc_res)
    test_acc = np.mean(test_acc_res) 
    
    #return test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject
    """수정 필요"""
    return test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject, y_valid_true



# Run function, Split data and and cross validation. This needs to be modified

In [8]:
def run(train_data_processed, test_data_processed, fold, Num_feat, features):
    name_test = test_data_processed['subjectkey'].values
    print("-------------------------------Training Begining-------------------------------")
    n_ = [4,8,16]
    lr_ = [2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 1e-4]
    w_ = [0.01, 0.001, 0.0001]
    g_ = [0.95, 0.99, 0.9]
    ss_ = [10, 20, 30]
    all_ = [n_, lr_, w_, g_, ss_]
    h_space = [s for s in itertools.product(*all_)]
    
    # Start training
    max_hy = find_bestpar(fold, train_data_processed, test_data_processed, features)
    
    # if you want to just test the code, you should use this
    #max_hy = h_space[0]
    #print("Found maximum hyperparmeter, now work with that")
    
    print("-------------------------------Testing Begining-------------------------------")
    test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject, y_valid_true= bestpar_tuning(fold, 
                                                                                        train_data_processed, 
                                                                                        test_data_processed, 
                                                                                        max_hy, 
                                                                                        features)
    
    #print("-------------------------------Important Feature-------------------------------")
    import_feat=feature(Num_feat, clf_res, test_data_processed, features)
    #import_feat=0
    #preds_val_prob = clf.predict_proba(X_valid)

    return test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, import_feat, name_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject, y_valid_true


In [9]:
"""해당 버전 추가 코드"""
def save_prob_with_true(model):
    modeltype= "" 
    combined_model=pd.DataFrame({f"subjectkey": model.y_test_subject[0], f"Y_{modeltype}":model.Y_test, f"preds_prob_{modeltype}" :model.preds_prob[:,1]} )
    combined_model.to_csv(f"combined_forROC_{modeltype}_.csv")
    #mdoel.import_feat_every.to_csv(f"{modeltype}_features.csv")

    return combined_model
def save_prob_with_true_valid(model):
    modeltype= "" 
    
    combined_model=pd.DataFrame({f"subjectkey": list(itertools.chain(*model.y_valid_subject)), f"Y_{modeltype}":list(itertools.chain(*model.y_valid_true)), f"preds_prob_{modeltype}" :list(itertools.chain(*model.y_valid_pred) )})
    combined_model.to_csv(f"combined_forROC_{modeltype}_.csv")
    #model.import_feat_every.to_csv(f"{modeltype}_features.csv")

    return combined_model

# Main code 

In [10]:
class model():
    def __init__(self, train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features):
        test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, import_feat, name_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject, y_valid_true = run(train_data_processed,
                                                                              test_data_processed,
                                                                              Num_FOLDS, 
                                                                              Num_feat, 
                                                                              features)
    
    
        self.train_data_processed = train_data_processed
        self.test_auc = test_auc
        self.test_acc = test_acc
        self.valid_result = valid_result
        self.clf_res = clf_res 
        self.preds_prob = preds_prob 
        self.X_test = X_test
        self.Y_test = Y_test
        self.import_feat =  import_feat
        self.name_test = name_test
        self.features = features
        self.y_valid_pred =y_valid_pred
        self.y_test_pred = y_test_pred
        self.y_valid_subject= y_valid_subject
        self.y_test_subject = y_test_subject
        self.y_valid_true = y_valid_true
        """해당 버전 추가 코드"""        
        test_prob_result = save_prob_with_true(self)
        valid_prob_result = save_prob_with_true_valid(self)

        """해당 버전 추가 코드"""
        self.test_prob_result= test_prob_result
        self.valid_prob_result = valid_prob_result        

In [11]:

train_data_processed, test_data_processed = preprocessing (train_data, test_data, Num_FOLDS)

features_rsf_sst_N_mid = [col for col in train_data_processed.columns if col in psychosocial + rsf_mri+sstfmri+Nbackfmri+midfmri]
features_rsf_sst_N = [col for col in train_data_processed.columns if col in psychosocial + rsf_mri+sstfmri+Nbackfmri]
features_rsf_sst = [col for col in train_data_processed.columns if col in psychosocial + rsf_mri+sstfmri]
features_rsf_N = [col for col in train_data_processed.columns if col in psychosocial + rsf_mri+Nbackfmri]

features_N_sst = [col for col in train_data_processed.columns if col in psychosocial + Nbackfmri+sstfmri]

# features_rsf_only = [col for col in train_data_processed.columns if col in psychosocial + rsf_mri]
# features_rsf_sst_N_mid = [col for col in train_data_processed.columns if col in psychosocial + rsf_mri+sstfmri+Nbackfmri+midfmri]
# features_rsf_sst_N = [col for col in train_data_processed.columns if col in psychosocial + rsf_mri+sstfmri+Nbackfmri]
# features_rsf_sst = [col for col in train_data_processed.columns if col in psychosocial + rsf_mri+sstfmri]

done preprocessing


In [12]:
rs_sst_N_midfMRI = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_rsf_sst_N_mid)
print("rs_sst_N_midfMRI.done")
rs_sst_NfMRI = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_rsf_sst_N)
print("rs_sst_NfMRI.done")
rsf_sstfMRI = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_rsf_sst)

-------------------------------Training Begining-------------------------------


Device used : cuda

Early stopping occured at epoch 22 with best_epoch = 2 and best_valid_auc = 0.70196
Best weights from best epoch are automatically used!
[  1/   5]  Valid score: 0.701961 Test AUC: 0.548% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 21 with best_epoch = 1 and best_valid_auc = 0.78431
Best weights from best epoch are automatically used!
[  2/   5]  Valid score: 0.784314 Test AUC: 0.381% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 27 with best_epoch = 7 and best_valid_auc = 0.76427
Best weights from best epoch are automatically used!
[  3/   5]  Valid score: 0.764270 Test AUC: 0.571% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 23 with best_epoch = 3 and best_valid_auc = 0.75098
Best weights from best epoch are automatically used!
[  4/   5]  Valid score: 0.750980 Test AUC: 0.857% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 44 with best_epoch = 24 and best_valid_auc = 0.9

Device used : cuda

Early stopping occured at epoch 44 with best_epoch = 24 and best_valid_auc = 0.72244
Best weights from best epoch are automatically used!
[  1/   5]  Valid score: 0.722440 Test AUC: 0.405% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 20 with best_epoch = 0 and best_valid_auc = 0.75185
Best weights from best epoch are automatically used!
[  2/   5]  Valid score: 0.751852 Test AUC: 0.310% Test ACC: 0.462%
Device used : cuda

Early stopping occured at epoch 29 with best_epoch = 9 and best_valid_auc = 0.739
Best weights from best epoch are automatically used!
[  3/   5]  Valid score: 0.738998 Test AUC: 0.571% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 20 with best_epoch = 0 and best_valid_auc = 0.64368
Best weights from best epoch are automatically used!
[  4/   5]  Valid score: 0.643682 Test AUC: 0.595% Test ACC: 0.462%
Device used : cuda

Early stopping occured at epoch 69 with best_epoch = 49 and best_valid_auc = 0.82

Device used : cuda

Early stopping occured at epoch 32 with best_epoch = 12 and best_valid_auc = 0.68453
Best weights from best epoch are automatically used!
[  1/   5]  Valid score: 0.684532 Test AUC: 0.595% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 69 with best_epoch = 49 and best_valid_auc = 0.90131
Best weights from best epoch are automatically used!
[  2/   5]  Valid score: 0.901307 Test AUC: 0.524% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 27 with best_epoch = 7 and best_valid_auc = 0.61656
Best weights from best epoch are automatically used!
[  3/   5]  Valid score: 0.616558 Test AUC: 0.333% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 22 with best_epoch = 2 and best_valid_auc = 0.54423
Best weights from best epoch are automatically used!
[  4/   5]  Valid score: 0.544227 Test AUC: 0.286% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 47 with best_epoch = 27 and best_valid_auc = 0

In [13]:
rs_sst_N_midfMRI = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_rsf_sst_N_mid)
print("rs_sst_N_midfMRI.done")
rs_sst_NfMRI = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_rsf_sst_N)
print("rs_sst_NfMRI.done")
rsf_sstfMRI = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_rsf_sst)

rsf_NfMRI = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_rsf_N)

print("rsf_NfMRI.done")

N_sstfMRI = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_N_sst)
print("N_sstfMRI.done")


-------------------------------Training Begining-------------------------------


Device used : cuda

Early stopping occured at epoch 28 with best_epoch = 8 and best_valid_auc = 0.73442
Best weights from best epoch are automatically used!
[  1/   5]  Valid score: 0.734423 Test AUC: 0.714% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 27 with best_epoch = 7 and best_valid_auc = 0.65512
Best weights from best epoch are automatically used!
[  2/   5]  Valid score: 0.655120 Test AUC: 0.524% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 49 with best_epoch = 29 and best_valid_auc = 0.75076
Best weights from best epoch are automatically used!
[  3/   5]  Valid score: 0.750763 Test AUC: 0.238% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 39 with best_epoch = 19 and best_valid_auc = 0.79063
Best weights from best epoch are automatically used!
[  4/   5]  Valid score: 0.790632 Test AUC: 0.548% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 29 with best_epoch = 9 and best_valid_auc = 0.

Device used : cuda

Early stopping occured at epoch 22 with best_epoch = 2 and best_valid_auc = 0.6329
Best weights from best epoch are automatically used!
[  1/   5]  Valid score: 0.632898 Test AUC: 0.214% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 22 with best_epoch = 2 and best_valid_auc = 0.59346
Best weights from best epoch are automatically used!
[  2/   5]  Valid score: 0.593464 Test AUC: 0.405% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 25 with best_epoch = 5 and best_valid_auc = 0.75359
Best weights from best epoch are automatically used!
[  3/   5]  Valid score: 0.753595 Test AUC: 0.357% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 24 with best_epoch = 4 and best_valid_auc = 0.57277
Best weights from best epoch are automatically used!
[  4/   5]  Valid score: 0.572767 Test AUC: 0.786% Test ACC: 0.538%
Device used : cuda

Early stopping occured at epoch 20 with best_epoch = 0 and best_valid_auc = 0.620

In [14]:
tasklist= [rs_sst_N_midfMRI,rs_sst_NfMRI, rsf_sstfMRI,rsf_NfMRI, N_sstfMRI]

In [15]:
for model in tasklist:
    print(model.test_auc)
   

0.6476190476190476
0.7523809523809525
0.5904761904761905
0.6095238095238096
0.6904761904761905


In [17]:
for model in tasklist:
    print(model.valid_result)

0.7781485045247954
0.7614684618546316
0.7373632879784653
0.8251677464879081
0.7018639448994186


In [16]:
tasklist= [rs_sst_N_midfMRI,rs_sst_NfMRI, rsf_sstfMRI,rsf_NfMRI, N_sstfMRI]

import dill
exp_type = "SA"

with open(f'./{exp_type}/rs_sst_N_midfMRI.pkl', 'wb') as f:
    dill.dump(rs_sst_N_midfMRI, f)
with open(f'./{exp_type}/rs_sst_NfMRI.pkl', 'wb') as f:
    dill.dump(rs_sst_NfMRI, f)
    
with open(f'./{exp_type}/rsf_sstfMRI.pkl', 'wb') as f:
    dill.dump(rsf_sstfMRI, f)
with open(f'./{exp_type}/rsf_NfMRI.pkl', 'wb') as f:
    dill.dump(rsf_NfMRI, f)
with open(f'./{exp_type}/N_sstfMRI.pkl', 'wb') as f:
    dill.dump(N_sstfMRI, f)

In [25]:
with open('sstfMRI.pkl', 'rb') as f:
    sstfMRI = dill.load(f)
with open('NbackfMRI-Copy1.pkl', 'rb') as f:
    Nback_fMRI = dill.load(f)
with open('rsfMRI-Copy1.pkl', 'rb') as f:
    rsfMRI = dill.load(f)
with open('midfMRI-Copy1.pkl', 'rb') as f:
    midfMRI = dill.load(f)
    
features_newway = [col for col in train_data_processed.columns if col in sstfMRI.import_feat.index + midfMRI.import_feat.index + rsfMRI.import_feat.index+ Nback_fMRI.import_feat.index]


In [30]:
features_newway

[]

In [38]:
sstfMRI.import_feat.index

Index(['WithDep.CBCL',
       'tfmri_sst_all_correct.stop.vs.incorrect.stop_beta_cort.destrieux_s.pericallosal.rh',
       'tfmri_sst_all_correct.stop.vs.correct.go_beta_cort.destrieux_g.cingul.post.ventral.rh',
       'nihtbx_list_uncorrected',
       'tfmri_sst_all_any.stop.vs.correct.go_beta_cort.destrieux_s.orbital.lateral.lh',
       'tfmri_sst_all_correct.stop.vs.incorrect.stop_beta_cort.destrieux_g.temp.sup.plan.polar.lh',
       'tfmri_sst_all_correct.stop.vs.correct.go_beta_cort.destrieux_g.ins.lg.and.s.cent.ins.rh',
       'tfmri_sst_all_correct.go.vs.fixation_beta_cort.destrieux_g.temporal.middle.lh',
       'tfmri_sst_all_correct.stop.vs.correct.go_beta_cort.destrieux_s.occipital.ant.rh',
       'tfmri_sst_all_any.stop.vs.correct.go_beta_cort.destrieux_s.calcarine.rh',
       'tfmri_sst_all_correct.stop.vs.correct.go_beta_cort.destrieux_s.circular.insula.sup.rh',
       'nihtbx_picture_uncorrected',
       'tfmri_sst_all_incorrect.go.vs.incorrect.stop_beta_cort.destrieux_s.

In [27]:
train_data_processed

,Unnamed: 0,subjectkey,abcd_site,Suicidalideation,race.ethnicity_1,race.ethnicity_2,race.ethnicity_3,race.ethnicity_4,race.ethnicity_5,sex_1,...,tfmri_mid_all_antic.large.vs.small.loss_beta_cort.destrieux_s.pericallosal.rh,tfmri_mid_all_antic.large.vs.small.loss_beta_cort.destrieux_s.postcentral.rh,tfmri_mid_all_antic.large.vs.small.loss_beta_cort.destrieux_s.precentral.inf.part.rh,tfmri_mid_all_antic.large.vs.small.loss_beta_cort.destrieux_s.precentral.sup.part.rh,tfmri_mid_all_antic.large.vs.small.loss_beta_cort.destrieux_s.suborbital.rh,tfmri_mid_all_antic.large.vs.small.loss_beta_cort.destrieux_s.subparietal.rh,tfmri_mid_all_antic.large.vs.small.loss_beta_cort.destrieux_s.temporal.inf.rh,tfmri_mid_all_antic.large.vs.small.loss_beta_cort.destrieux_s.temporal.sup.rh,tfmri_mid_all_antic.large.vs.small.loss_beta_cort.destrieux_s.temporal.transverse.rh,kfold
0,2448,NDARINVHUL3V5Z7,9,0,1,0,0,0,0,1,...,-1.448862,-0.868742,0.120642,-0.893485,1.064481,-0.618244,0.576286,-0.149634,0.257284,3
1,3504,NDARINVT8U0418X,4,0,1,0,0,0,0,0,...,-0.883636,0.116826,-0.230057,0.344139,-0.181346,-0.862811,-0.429196,-0.127397,1.330653,0
2,752,NDARINV5B32UHDY,20,0,0,0,0,0,1,0,...,-2.029345,-1.705278,-0.985712,-0.876942,-1.265292,-1.905579,0.951202,-2.073431,-1.290674,2
3,195,NDARINV182YC0D9,20,0,1,0,0,0,0,0,...,1.033510,0.366408,-0.525111,0.997569,-0.156611,-0.807461,-0.480509,2.320566,-2.503975,4
4,1431,NDARINVA7E3XLG4,18,0,1,0,0,0,0,0,...,-0.444460,-0.439823,-0.174326,-0.507944,-0.125299,-0.334054,-0.232587,-0.224478,-0.262393,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4385,3656,NDARINVU830CVT7,2,0,1,0,0,0,0,0,...,0.510006,1.508396,1.012413,1.619136,-0.857796,0.470682,0.405699,1.202302,2.527103,1
4386,1662,NDARINVC25291CE,8,0,0,0,0,1,0,0,...,0.362920,1.129514,0.502777,-0.047108,0.367872,0.513542,0.072212,0.643060,0.995569,4
4387,2140,NDARINVFPEWC40T,4,0,1,0,0,0,0,1,...,-0.627679,-0.111001,0.511789,-0.138635,0.132057,-0.384605,0.243460,0.052561,-0.360041,2
4388,3780,NDARINVV1BV9PWF,6,0,1,0,0,0,0,0,...,-0.197099,-1.294147,-0.189654,-0.282320,-1.313362,0.543592,-0.030542,0.205934,-0.158896,3


In [26]:
features_newway

[]

In [21]:
rsf_SMRI.test_auc


0.7081999840777009

In [22]:
rsf_SMRI.valid_result

0.7626530194472878

In [23]:
rsf_DMRI.test_auc

0.6859843961468036

In [24]:
rsf_DMRI.valid_result

0.750221494370522

In [20]:
def show(model):
    print("test_auc:", round(model.test_auc,4))
    print("valid_auc:", round(model.valid_result,4))

In [27]:
show(sMRI)


test_auc: 0.6714
valid_auc: 0.7458


NameError: name 'dMRI' is not defined

In [47]:
sMRI.test_auc

0.7041875646843405

In [48]:
sMRI.valid_result

0.7644118730808598

In [37]:
dMRI.test_auc

0.6714194729718972

In [38]:
dMRI.valid_result

0.745781780962129

In [45]:

dMRI = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_dMRI)
print("dMRI.done")

-------------------------------Training Begining-------------------------------


Device used : cuda

Early stopping occured at epoch 41 with best_epoch = 21 and best_valid_auc = 0.74237
Best weights from best epoch are automatically used!
[  1/   5]  Valid score: 0.742371 Test AUC: 0.698% Test ACC: 0.502%
Device used : cuda

Early stopping occured at epoch 41 with best_epoch = 21 and best_valid_auc = 0.76625
Best weights from best epoch are automatically used!
[  2/   5]  Valid score: 0.766248 Test AUC: 0.700% Test ACC: 0.502%
Device used : cuda

Early stopping occured at epoch 42 with best_epoch = 22 and best_valid_auc = 0.76646
Best weights from best epoch are automatically used!
[  3/   5]  Valid score: 0.766461 Test AUC: 0.635% Test ACC: 0.502%
Device used : cuda

Early stopping occured at epoch 53 with best_epoch = 33 and best_valid_auc = 0.71718
Best weights from best epoch are automatically used!
[  4/   5]  Valid score: 0.717183 Test AUC: 0.627% Test ACC: 0.502%
Device used : cuda

Early stopping occured at epoch 38 with best_epoch = 18 and best_valid_auc =

In [49]:
features_S_D = [col for col in train_data_processed.columns if col in psychosocial + structural_mri+diffusion_mri]

S_D_MRI = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_S_D)
print("S_D_MRI.done")


-------------------------------Training Begining-------------------------------


Device used : cuda

Early stopping occured at epoch 48 with best_epoch = 28 and best_valid_auc = 0.71614
Best weights from best epoch are automatically used!
[  1/   5]  Valid score: 0.716135 Test AUC: 0.697% Test ACC: 0.508%
Device used : cuda

Early stopping occured at epoch 38 with best_epoch = 18 and best_valid_auc = 0.7393
Best weights from best epoch are automatically used!
[  2/   5]  Valid score: 0.739300 Test AUC: 0.686% Test ACC: 0.502%
Device used : cuda

Early stopping occured at epoch 35 with best_epoch = 15 and best_valid_auc = 0.76138
Best weights from best epoch are automatically used!
[  3/   5]  Valid score: 0.761384 Test AUC: 0.682% Test ACC: 0.502%
Device used : cuda

Early stopping occured at epoch 41 with best_epoch = 21 and best_valid_auc = 0.72409
Best weights from best epoch are automatically used!
[  4/   5]  Valid score: 0.724086 Test AUC: 0.739% Test ACC: 0.502%
Device used : cuda

Early stopping occured at epoch 39 with best_epoch = 19 and best_valid_auc = 

In [50]:
S_D_MRI.test_auc

0.6991959238914098

In [51]:
S_D_MRI.valid_result

0.7436593654042989

In [ ]:
print("rsfmri",rsfMRI.test_auc)
print("rs_sst_N_midfMRI",rs_sst_N_midfMRI.test_auc)
print("rs_sst_NfMRI",rs_sst_NfMRI.test_auc)
print("rsf_sstfMRI",rsf_sstfMRI.test_auc)

In [ ]:
features_mid_sst = [col for col in train_data_processed.columns if col in psychosocial + midfmri+ sstfmri]
features_mid_N= [col for col in train_data_processed.columns if col in psychosocial + midfmri+Nbackfmri]
features_sst_N = [col for col in train_data_processed.columns if col in psychosocial + sstfmri+Nbackfmri]


In [ ]:
mid_sstmri = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_mid_sst)
print("mid_sstmri.done")

mid_Nfmri = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_mid_N)
print("mid_Nfmri.done")

sstf_Nmri = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_sst_N)
print("sstf_Nmri.done")


In [ ]:
print("mid_sstmri",mid_sstmri.test_auc)
print("mid_Nfmri",mid_Nfmri.test_auc)
print("sstf_Nmri",sstf_Nmri.test_auc)


In [ ]:
features_rsf_only_nopsy = [col for col in train_data_processed.columns if col in rsf_mri]
features_rsf_sst_N_mid_nopsy = [col for col in train_data_processed.columns if col in rsf_mri+sstfmri+Nbackfmri+midfmri]
features_rsf_sst_N_nopsy = [col for col in train_data_processed.columns if col in rsf_mri+sstfmri+Nbackfmri]
features_rsf_sst_nopsy = [col for col in train_data_processed.columns if col in  rsf_mri+sstfmri]

In [ ]:
rsfMRI_nopsy = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_rsf_only_nopsy)
print("rsfMRI_nopsy.done")
rs_sst_N_midfMRI_nopsy = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_rsf_sst_N_mid_nopsy)
print("rs_sst_N_midfMRI_nopsy.done")
rs_sst_NfMRI_nopsy = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_rsf_sst_N_nopsy)
print("rs_sst_NfMRI_nopsy.done")
rsf_sstfMRI_nopsy = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features_rsf_sst_nopsy)

In [ ]:
print("rsfMRI_nopsy",rsfMRI_nopsy.test_auc)
print("rs_sst_N_midfMRI_nopsy",rs_sst_N_midfMRI_nopsy.test_auc)
print("rs_sst_NfMRI_nopsy",rs_sst_NfMRI_nopsy.test_auc)
print("rsf_sstfMRI_nopsy",rsf_sstfMRI_nopsy.test_auc)

# For drawing ROC curve graph

In [ ]:
def rocvis(true , prob , label ) :
    from sklearn.metrics import roc_curve
    if type(true[0]) == str :
        from sklearn.preprocessing import LabelEncoder
        le = LabelEncoder()
        true = le.fit_transform(true)
    else :
        pass
    fpr, tpr, thresholds = roc_curve(true, prob)
    plt.plot(fpr, tpr, marker='.', label = label  )

# Draw ROC curve

In [ ]:
fig , ax = plt.subplots(figsize= (20,10))
plt.plot([0, 1], [0, 1], linestyle='--')
rocvis(rsfMRI.Y_test , rsfMRI.preds_prob[:,1] , f"rsfMRI:{round(rsfMRI.test_auc*100, 2)}%")
rocvis(rs_sst_N_midfMRI.Y_test , rs_sst_N_midfMRI.preds_prob[:,1] , f"rs_sst_N_midfMRI:{round(rs_sst_N_midfMRI.test_auc*100, 2)}%")
rocvis(rs_sst_NfMRI.Y_test , rs_sst_NfMRI.preds_prob[:,1] , f"rs_sst_NfMRI:{round(rs_sst_NfMRI.test_auc*100, 2)}%")
rocvis(rsfMRI.Y_test , rsf_sstfMRI.preds_prob[:,1] , f"rsf_sstfMRI:{round(rsf_sstfMRI.test_auc*100, 2)}%")

dataset_name="3mris"

plt.legend(fontsize = 40)
plt.title(dataset_name, fontsize= 50)
plt.xlabel("FP rate", fontsize =30)
plt.ylabel("TP rate", fontsize =30)
plt.xticks(size = 30)
plt.yticks(size = 30)

plt.tight_layout()
plt.savefig(f'{dataset_name}_ROC.png')

# For model interpretation. 

"{modeltype}_features.csv" and "combined_forROC_{modeltype}_.csv" will be saved 



In [ ]:
def save_prob_with_true(model, modeltype):
   
    combined_model=pd.DataFrame({f"subjectkey": model.name_test, f"Y_{modeltype}":model.Y_test, f"preds_prob_{modeltype}" :model.preds_prob[:,1]} )
    combined_model.to_csv(f"combined_forROC_{modeltype}_.csv")
    mdoel.import_feat_every.to_csv(f"{modeltype}_features.csv")

    return combined_model


In [ ]:
model_inter_rsfMRI = save_prob_with_true(rsfMRI, "rsfMRI")
model_inter_rs_sst_N_midfMRI = save_prob_with_true(rs_sst_N_midfMRI, "rs_sst_N_midfMRI")
model_inter_rs_sst_NfMRI = save_prob_with_true(rs_sst_NfMRI, "rs_sst_NfMRI")
model_inter_rsf_sstfMRI = save_prob_with_true(rsf_sstfMRI, "rsf_sstfMRI")


# Model save 

In [ ]:
with open('./rsfMRI.pkl', 'wb') as f:
    dill.dump(rsfMRI, f)

In [ ]:
import dill

with open('./rsfMRI.pkl', 'wb') as f:
    dill.dump(rsfMRI, f)
with open('./rs_sst_N_midfMRI.pkl', 'wb') as f:
    dill.dump(rs_sst_N_midfMRI, f)
with open('./rs_sst_NfMRI.pkl', 'wb') as f:
    dill.dump(rs_sst_NfMRI, f)
with open('./rsf_sstfMRI.pkl', 'wb') as f:
    dill.dump(rsf_sstfMRI, f)

#with open('./rsfMRI.pkl', 'rb') as f:
#    data = dill.load(f)

# Draw AUC per epochs and loss per epochs

feature_names=["Combined", "psychosocial", "MRI"]
clfs = [clf_all, clf_pheno, clf_mri ]
simple_name = "enbackfMRI"

def error_plot(clf):
    plt.plot(clf.history['train_auc'])
    plt.plot(clf.history['valid_auc'])

for i in range(3):
    error_plot(clfs[i])

plt.legend([f'train_{feature_names[0]}', f'valid_{feature_names[0]}', 
           f'train_{feature_names[1]}', f'valid_{feature_names[1]}',
           f'train_{feature_names[2]}', f'valid_{feature_names[2]}'])
plt.title(f'{simple_name}')
plt.xlabel('epochs')
plt.ylabel('AUC')
plt.tight_layout()

plt.savefig(f'AUC_per_epochs {simple_name}.png')

plt.show()          


feature_names=["combined", "psychosocial", "MRI"]
clfs = [clf_all, clf_pheno, clf_mri ]

def error_plot(clf):
    plt.plot(clf.history['loss'])

for i in range(3):
    error_plot(clfs[i])

plt.legend([f'{feature_names[0]}', 
           f'{feature_names[1]}',
           f'{feature_names[2]}'])
plt.title(f'{simple_name}')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.tight_layout()

plt.savefig(f'loss_per_epochs {simple_name}.png')

plt.show()